In [24]:
# Resources
import numpy as np
import sqlite3
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract
from sqlalchemy import select
from sqlalchemy.orm import join
from sqlalchemy.sql import func
from sqlalchemy import or_
from sqlalchemy import inspect

In [25]:
##################################
# Database Setup
##################################

# https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite
# Module 9 Challenge
engine = create_engine(r'sqlite:///../../data/LIFE_EXPECTANCY_DB.db')

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Happiness = Base.classes.HAPPINESS_TABLE
Expectancy = Base.classes.EXPECTANCY_TABLE

In [26]:
# Tables in database
Base.classes.keys()

['EXPECTANCY_TABLE', 'HAPPINESS_TABLE']

In [27]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [28]:
inst = inspect(Expectancy)
Expectancy_columns = [c_attr.key for c_attr in inst.mapper.column_attrs]

In [29]:
# Columns of EXPECTANCY_TABLE
print(Expectancy_columns)

['ID', 'COUNTRY', 'YEAR', 'STATUS', 'EXPECTANCY', 'MORTALITY', 'INFANT_DEATH', 'ALCOHOL', 'EXPENDITURE_PERCENT', 'HEPATITUS_B', 'MEASLES', 'BMI', 'UNDER_FIVE_DEATH', 'POLIIO', 'EXPENDITURE_TOTAL', 'DIPHTHERIA', 'HIV_AIDS', 'GDP', 'POPULATION', 'THIN_1TO19_YR', 'THIN_5TO9_YR', 'INC_COMPOSITION', 'SCHOOLING']


In [30]:
inst = inspect(Happiness)
Happiness_columns = [c_attr.key for c_attr in inst.mapper.column_attrs]

In [31]:
# Columns of HAPPINESS_TABLE
print(Happiness_columns)

['ID', 'COUNTRY', 'REGION', 'HAPPINESS_SCORE', 'HAPPINESS_RANK', 'LOW_CONF', 'HIGH_CONF', 'STANDARD_ERROR', 'ECONOMY', 'FAMILY', 'HEALTH', 'FREEDOM', 'TRUST', 'GENEROSITY', 'DYSTOPIA', 'YEAR']


In [32]:
expect_results = []

# Query contents of EXPECTANCY_TABLE to list
expect_results = session.query(
    Expectancy.ID,
    Expectancy.COUNTRY,
    Expectancy.YEAR,
    Expectancy.STATUS,
    Expectancy.EXPECTANCY,
    Expectancy.MORTALITY,
    Expectancy.INFANT_DEATH,
    Expectancy.ALCOHOL,
    Expectancy.EXPENDITURE_PERCENT,
    Expectancy.HEPATITUS_B,
    Expectancy.MEASLES,
    Expectancy.BMI,
    Expectancy.UNDER_FIVE_DEATH,
    Expectancy.POLIIO,
    Expectancy.EXPENDITURE_TOTAL,
    Expectancy.DIPHTHERIA,
    Expectancy.HIV_AIDS,
    Expectancy.GDP,
    Expectancy.POPULATION,
    Expectancy.THIN_1TO19_YR,
    Expectancy.THIN_5TO9_YR,
    Expectancy.INC_COMPOSITION,
    Expectancy.SCHOOLING
    ).all()

#for temp_result in expect_results:
#    print(temp_result)

In [33]:
# Convert Expectancy list to dataframe
expect_df = pd.DataFrame(expect_results, columns=Expectancy_columns)
print(expect_df)

        ID      COUNTRY  YEAR      STATUS EXPECTANCY MORTALITY  INFANT_DEATH  \
0        1  Afghanistan  2015  Developing         65       263          62.0   
1        2  Afghanistan  2014  Developing       59.9       271          64.0   
2        3  Afghanistan  2013  Developing       59.9       268          66.0   
3        4  Afghanistan  2012  Developing       59.5       272          69.0   
4        5  Afghanistan  2011  Developing       59.2       275          71.0   
...    ...          ...   ...         ...        ...       ...           ...   
2933  2934     Zimbabwe  2004  Developing       44.3       723          27.0   
2934  2935     Zimbabwe  2003  Developing       44.5       715          26.0   
2935  2936     Zimbabwe  2002  Developing       44.8        73          25.0   
2936  2937     Zimbabwe  2001  Developing       45.3       686          25.0   
2937  2938     Zimbabwe  2000  Developing         46       665          24.0   

     ALCOHOL  EXPENDITURE_PERCENT HEPAT

In [34]:
happy_results = []

# Query contents of HAPPINESS_TABLE to list
happy_results = session.query(
    Happiness.ID,
    Happiness.COUNTRY,
    Happiness.REGION,
    Happiness.HAPPINESS_SCORE,
    Happiness.HAPPINESS_RANK,
    Happiness.LOW_CONF,
    Happiness.HIGH_CONF,    
    Happiness.STANDARD_ERROR,
    Happiness.ECONOMY,
    Happiness.FAMILY,
    Happiness.HEALTH,
    Happiness.FREEDOM,
    Happiness.TRUST,
    Happiness.GENEROSITY,
    Happiness.DYSTOPIA,
    Happiness.YEAR
    ).all()

#for temp_result in happy_results:
#    print(temp_result)

In [35]:
# Convert Happiness list to dataframe
happy_df = pd.DataFrame(happy_results, columns=Happiness_columns)
happy_df = happy_df.rename(columns={"HAPPINESS_SCORE":"HAPPINESS_RANK","HAPPINESS_RANK":"HAPPINESS_SCORE"})

print(happy_df)

      ID                   COUNTRY          REGION  HAPPINESS_RANK  \
0      1               Switzerland  Western Europe             1.0   
1      2                   Iceland  Western Europe             2.0   
2      3                   Denmark  Western Europe             3.0   
3      4                    Norway  Western Europe             4.0   
4      5                    Canada   North America             5.0   
..   ...                       ...             ...             ...   
777  778                    Rwanda                           152.0   
778  779                  Tanzania                           153.0   
779  780               Afghanistan                           154.0   
780  781  Central African Republic                           155.0   
781  782               South Sudan                           156.0   

     HAPPINESS_SCORE LOW_CONF HIGH_CONF STANDARD_ERROR  ECONOMY   FAMILY  \
0              7.587                           0.03411  1.39651  1.34951   
1      

In [36]:
# INNER JOIN between Expectancy & Happiness data with filter on Expectancy data to only view 2015 year.

happy_expect_join = []

happy_expect_join_columns = Expectancy_columns + Happiness_columns

happy_expect_join = session.query(
    Expectancy.ID.label("E_ID"),
    Expectancy.COUNTRY,
    Expectancy.YEAR,
    Expectancy.STATUS,
    Expectancy.EXPECTANCY,
    Expectancy.MORTALITY,
    Expectancy.INFANT_DEATH,
    Expectancy.ALCOHOL,
    Expectancy.EXPENDITURE_PERCENT,
    Expectancy.HEPATITUS_B,
    Expectancy.MEASLES,
    Expectancy.BMI,
    Expectancy.UNDER_FIVE_DEATH,
    Expectancy.POLIIO,
    Expectancy.EXPENDITURE_TOTAL,
    Expectancy.DIPHTHERIA,
    Expectancy.HIV_AIDS,
    Expectancy.GDP,
    Expectancy.POPULATION,
    Expectancy.THIN_1TO19_YR,
    Expectancy.THIN_5TO9_YR,
    Expectancy.INC_COMPOSITION,
    Expectancy.SCHOOLING,
    Happiness.ID,
    Happiness.COUNTRY,
    Happiness.REGION,
    Happiness.HAPPINESS_SCORE,
    Happiness.HAPPINESS_RANK,
    Happiness.LOW_CONF,
    Happiness.HIGH_CONF,    
    Happiness.STANDARD_ERROR,
    Happiness.ECONOMY,
    Happiness.FAMILY,
    Happiness.HEALTH,
    Happiness.FREEDOM,
    Happiness.TRUST,
    Happiness.GENEROSITY,
    Happiness.DYSTOPIA,
    Happiness.YEAR
).join(Happiness, func.lower(Expectancy.COUNTRY) == func.lower(Happiness.COUNTRY)
).filter(Expectancy.YEAR == '2015'
).filter(Happiness.YEAR == '2015').all()

happy_expect_join_df = pd.DataFrame(happy_expect_join, columns=happy_expect_join_columns)

print(happy_expect_join_df)

       ID      COUNTRY  YEAR      STATUS  EXPECTANCY  MORTALITY  INFANT_DEATH  \
0       1  Afghanistan  2015  Developing        65.0      263.0          62.0   
1      17      Albania  2015  Developing        77.8       74.0           0.0   
2      33      Algeria  2015  Developing        75.6       19.0          21.0   
3      49       Angola  2015  Developing        52.4      335.0          66.0   
4      81    Argentina  2015  Developing        76.3      116.0           8.0   
..    ...          ...   ...         ...         ...        ...           ...   
130  2811      Uruguay  2015  Developing        77.0      116.0           0.0   
131  2827   Uzbekistan  2015  Developing        69.4      184.0          15.0   
132  2891        Yemen  2015  Developing        65.7      224.0          37.0   
133  2907       Zambia  2015  Developing        61.8       33.0          27.0   
134  2923     Zimbabwe  2015  Developing        67.0      336.0          22.0   

    ALCOHOL  EXPENDITURE_PE

In [37]:
# Columns of the joined data between Expectancy & Happiness

for col in happy_expect_join_df.columns: 
    print(col) 

ID
COUNTRY
YEAR
STATUS
EXPECTANCY
MORTALITY
INFANT_DEATH
ALCOHOL
EXPENDITURE_PERCENT
HEPATITUS_B
MEASLES
BMI
UNDER_FIVE_DEATH
POLIIO
EXPENDITURE_TOTAL
DIPHTHERIA
HIV_AIDS
GDP
POPULATION
THIN_1TO19_YR
THIN_5TO9_YR
INC_COMPOSITION
SCHOOLING
ID
COUNTRY
REGION
HAPPINESS_SCORE
HAPPINESS_RANK
LOW_CONF
HIGH_CONF
STANDARD_ERROR
ECONOMY
FAMILY
HEALTH
FREEDOM
TRUST
GENEROSITY
DYSTOPIA
YEAR


In [38]:
# Check for NULL values in Happy Dataframe
for column in happy_df.columns:
    print(f'Column {column} has {happy_df[column].isnull().sum()} null values')

Column ID has 0 null values
Column COUNTRY has 0 null values
Column REGION has 0 null values
Column HAPPINESS_RANK has 0 null values
Column HAPPINESS_SCORE has 0 null values
Column LOW_CONF has 0 null values
Column HIGH_CONF has 0 null values
Column STANDARD_ERROR has 0 null values
Column ECONOMY has 0 null values
Column FAMILY has 0 null values
Column HEALTH has 0 null values
Column FREEDOM has 0 null values
Column TRUST has 0 null values
Column GENEROSITY has 0 null values
Column DYSTOPIA has 0 null values
Column YEAR has 0 null values


In [39]:
# Check for NULL values in Expect Dataframe
for column in expect_df.columns:
    print(f'Column {column} has {expect_df[column].isnull().sum()} null values')

Column ID has 0 null values
Column COUNTRY has 0 null values
Column YEAR has 0 null values
Column STATUS has 0 null values
Column EXPECTANCY has 0 null values
Column MORTALITY has 0 null values
Column INFANT_DEATH has 0 null values
Column ALCOHOL has 0 null values
Column EXPENDITURE_PERCENT has 0 null values
Column HEPATITUS_B has 0 null values
Column MEASLES has 0 null values
Column BMI has 0 null values
Column UNDER_FIVE_DEATH has 0 null values
Column POLIIO has 0 null values
Column EXPENDITURE_TOTAL has 0 null values
Column DIPHTHERIA has 0 null values
Column HIV_AIDS has 0 null values
Column GDP has 0 null values
Column POPULATION has 0 null values
Column THIN_1TO19_YR has 0 null values
Column THIN_5TO9_YR has 0 null values
Column INC_COMPOSITION has 0 null values
Column SCHOOLING has 0 null values


In [40]:
# Close DB Session
session.close()